
### Convolutional Neural Networks Modelling

-Build and train a convolutional Neuro network to detect if there is a dog or cat in the dog

##### Understanding the [Dataset](../dataFiles/Churn_Modelling.csv)



In [9]:
# Install tensorflow
!pip install --upgrade tensorflow

Defaulting to user installation because normal site-packages is not writeable


#### Importing the Libraries

In [7]:
import tensorflow as tf
# module for preprocessing images, needed for image data generator
from keras.preprocessing.image import ImageDataGenerator

In [12]:
tf.__version__

'2.13.0'

### Part 1 - Data preprocessing

***Split original images into training and test set***

In [ ]:
import os
import random
import shutil
#define folder paths
dogs_folder = '../dataFiles/PetImages/Dog/'
cats_folder = '../dataFiles/PetImages/Cat/'
train_folder = '../dataFiles/train'
test_folder = '../dataFiles/test'
#define split ratio
split_ratio = 0.7

#list the files in in the dog and cat folders
dog_files = os.listdir(dogs_folder)
cat_files = os.listdir(cats_folder)
#shuffle filenames - remove file arrangement biases e.g. breed sorting
random.shuffle(dog_files)
random.shuffle(cat_files)
#calculate split index i.e. position where the split will occur
dog_split_idx = int(len(dog_files) * split_ratio)
cat_split_idx = int(len(cat_files) * split_ratio)
print(f'Dog Split index: {dog_split_idx}| Cat Spit Index: {cat_split_idx }')
# Split the files into training and test sets
dog_train = dog_files[:dog_split_idx]
dog_test = dog_files[dog_split_idx:]
cat_train = cat_files[:cat_split_idx]
cat_test = cat_files[cat_split_idx:]

# Create the training and test directories if they don't exist
os.makedirs(os.path.join(train_folder, 'dog'), exist_ok=True)
os.makedirs(os.path.join(train_folder, 'cat'), exist_ok=True)
os.makedirs(os.path.join(test_folder, 'cat'), exist_ok=True)
os.makedirs(os.path.join(test_folder, 'dog'), exist_ok=True)

# Copy dog images to training and test directories
for file in dog_train:
    src_path = os.path.join(dogs_folder, file)
    dest_path = os.path.join(train_folder, 'dog', file)
    shutil.copy(src_path, dest_path)

for file in dog_test:
    src_path = os.path.join(dogs_folder, file)
    dest_path = os.path.join(test_folder, 'dog', file)
    shutil.copy(src_path, dest_path)

# Copy cat images to training and test directories
for file in cat_train:
    src_path = os.path.join(cats_folder, file)
    dest_path = os.path.join(train_folder, 'cat', file)
    shutil.copy(src_path, dest_path)

for file in cat_test:
    src_path = os.path.join(cats_folder, file)
    dest_path = os.path.join(test_folder, 'cat', file)
    shutil.copy(src_path, dest_path)

print("Data split and copied successfully.")


#### Processing the Training Set
- Apply transformations on the images of the training set to avoid over fitting if we ***DONOT*** apply transformations when training the CNN on the test set we will have huge differences between the accuracy on the training set and the one on the evaluation set. Higher accuracy on the training set i.e. 98% and much lower accuracy on the test set  is referred to as **Over fitting**.
- applying transformations avoids over fitting.
- Geometrical transformations could include - Zoom e.g. Zoom in and Zoom out, rotations, transvection i.e. shifting some of the pixels , horizontal flips - this processing is referred to as ***image augmentation*** - ensures that the CNN is not over trained on the existing images.



In [13]:
#instance of image of the data generator class
#rescale will apply feature scaling on the pixels
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

#connect the instance and connect it to the dataset
#Parameters - target size of 64 gives reasonable training time, Class mode is binary because outcome is binary i.e. cat or dog
training_set= train_datagen.flow_from_directory(
        '../dataFiles/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

#### Processing the test set

In [30]:
# No transformations should be performed on the test images
# However they must be re-scaled / feature/pixel scaled
test_datagen = ImageDataGenerator(rescale=1./255)

#importing the test set images into the notebook
test_set = test_datagen.flow_from_directory(
        '../dataFiles/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 13237 images belonging to 2 classes.


### Part 2 - Building the CNN

- Initialise the CNN as a sequence of layers
- Add the input layer and the first hidden layer composed a certian number of neurons
- add a second hidden layer to build a deep learning model instead of a shallow learning model
- finally add the output layer which will predict our output


***Initialising the CNN***

In [32]:
#create an artificial neural network as a sequence of layers
cnn = tf.keras.models.Sequential()


***Step 1: Convolution***
- add the Convolutional layer as a object of the Conv2D class
- Filters / Kernels - number of feature detectors - classic architecture uses 32 filters in the first and second convolution layers
- Kernel Size - size of the feature detector i.e. square array of feature detector
- Activation parameter - needs to be ***'Relu'*** which is the rectifier function - required as long as we have not reached the last layer
- input_shape - for the very first layer you need to put the input layer.  Input shape is (64,64, 3) because we are working with coloured images (RGB) for black and white images we would have (64,64, 1) first two parameter correspond to target size used in data preprocessing

In [33]:
#add a convolutional layer to the CNN
cnn.add(tf.keras.layers.Conv2D(filters =32, kernel_size=3, activation='relu', input_shape=[64,64, 3]))

***Step 2 Pooling***
- adding the Pooling layers
- Max pooling layer
- Pool_size illustration
- stride - refers to the stride
- padding is the parameter used when we get to the end of the feature map when max pooling -keep the default value

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, stride=2))

***Adding a second Convolution Layer***


In [35]:
#second convolution layer - removing the input shape parameter as we are now on the second convolution layer
cnn.add(tf.keras.layers.Conv2D(filters =32, kernel_size=3, activation='relu' ))
#pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

***Step 3 Flattening***
- flattening the results of all convolutions and poolings into a 1D vector which becomes the input of a future fully connected ANN
- Using the Flatten Class which takens no parameters


In [36]:
cnn.add(tf.keras.layers.Flatten())

***Step 4 Full Connection***
- units - number of hidden neurons - 128 hidden neurons as the problem is more complex
- activation function - use rectifier as we have not reached the final output layer


In [38]:
# adding a fully connected layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))


***Step 5: Output layer***
- number of neurons in the output layer is 1 as we are doing binary classification
- activation function for the output layer should be ***Sigmoid*** and if we has more classes in the output we would use ***softmax***

In [40]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Part 3 - Training the ANN

***Compiling the ANN***


***Training the CNN on the Training Dataset and evaluating it on the Test set***


### Part 4: Making a single prediction